In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [3]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 0us/step


'C:\\Users\\Duongkx\\.keras\\datasets'

In [4]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

In [5]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [6]:
all_labeled_data = labeled_data_sets[0]
for labeled_data in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [7]:
for ex in all_labeled_data.take(5):
    print(ex)

(<tf.Tensor: id=74, shape=(), dtype=string, numpy=b'Achaeans to free his daughter, and brought with him a great ransom:'>, <tf.Tensor: id=75, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=76, shape=(), dtype=string, numpy=b'The King, and roused his herald. Hermes yoked'>, <tf.Tensor: id=77, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=78, shape=(), dtype=string, numpy=b'upon him. May you prosper with the ransom and reach your own land in'>, <tf.Tensor: id=79, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=80, shape=(), dtype=string, numpy=b"Of Priam, wedded to Antenor's son,">, <tf.Tensor: id=81, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=82, shape=(), dtype=string, numpy=b'Achaeans, the one thrown into confusion, and the others driving them'>, <tf.Tensor: id=83, shape=(), dtype=int64, numpy=2>)


In [8]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
    
vocab_size = len(vocabulary_set)
vocab_size

14436

In [9]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [10]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'Achaeans to free his daughter, and brought with him a great ransom:'


In [11]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[3479, 5138, 11109, 6774, 12263, 6896, 8961, 2298, 1902, 9131, 5136, 12922]


In [12]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [13]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [16]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: id=477614, shape=(16,), dtype=int64, numpy=
 array([ 3237, 14203,  9223,  1261,  6774,  6378,  4391,   913, 10345,
        11343, 10910,  6896,  9187,  6133,  9223, 12069], dtype=int64)>,
 <tf.Tensor: id=477618, shape=(), dtype=int64, numpy=2>)

In [17]:
vocab_size +=1

In [18]:
model = tf.keras.Sequential()

In [19]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [20]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [21]:
for units in [64,64]:
    model.add(tf.keras.layers.Dense(units, activation='relu'))

model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
601/601 [==============================].0991 - accuracy: 0.25 - 20s 10s/step - loss: 1.0964 - accuracy: 0.32 - 20s 7s/step - loss: 1.0920 - accuracy: 0.4115 - 20s 5s/step - loss: 1.0878 - accuracy: 0.515 - 20s 4s/step - loss: 1.0830 - accuracy: 0.550 - 20s 3s/step - loss: 1.0781 - accuracy: 0.612 - 20s 3s/step - loss: 1.0731 - accuracy: 0.654 - 20s 2s/step - loss: 1.0672 - accuracy: 0.681 - 20s 2s/step - loss: 1.0605 - accuracy: 0.710 - 20s 2s/step - loss: 1.0534 - accuracy: 0.731 - 20s 2s/step - loss: 1.0449 - accuracy: 0.751 - 20s 2s/step - loss: 1.0354 - accuracy: 0.763 - 20s 2s/step - loss: 1.0254 - accuracy: 0.768 - 20s 1s/step - loss: 1.0147 - accuracy: 0.766 - 20s 1s/step - loss: 1.0038 - accuracy: 0.754 - 20s 1s/step - loss: 0.9915 - accuracy: 0.735 - 20s 1s/step - loss: 0.9796 - accuracy: 0.718 - 20s 1s/step - loss: 0.9662 - accuracy: 0.706 - 20s 1s/step - loss: 0.9529 - accuracy: 0.689 - 20s 1s/step - loss: 0.9390 - accuracy: 0.675 - 20s 966ms/step - loss: 0.9265 -

In [25]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

601/601 [==============================].9051e-04 - accuracy: 1.00 - 12s 6s/step - loss: 3.1261e-04 - accuracy: 1.0000 - 12s 4s/step - loss: 6.0145e-04 - accuracy: 1.000 - 12s 3s/step - loss: 4.8591e-04 - accuracy: 1.000 - 12s 2s/step - loss: 0.0015 - accuracy: 1.0000    - 12s 2s/step - loss: 0.0014 - accuracy: 1.000 - 12s 2s/step - loss: 0.0013 - accuracy: 1.000 - 12s 2s/step - loss: 0.0012 - accuracy: 1.000 - 12s 1s/step - loss: 0.0011 - accuracy: 1.000 - 12s 1s/step - loss: 0.0011 - accuracy: 1.000 - 12s 1s/step - loss: 0.0011 - accuracy: 1.000 - 12s 1s/step - loss: 0.0011 - accuracy: 1.000 - 12s 942ms/step - loss: 0.0015 - accuracy: 1.00 - 12s 876ms/step - loss: 0.0014 - accuracy: 1.00 - 12s 819ms/step - loss: 0.0014 - accuracy: 1.00 - 12s 769ms/step - loss: 0.0014 - accuracy: 1.00 - 12s 724ms/step - loss: 0.0014 - accuracy: 1.00 - 12s 685ms/step - loss: 0.0013 - accuracy: 1.00 - 12s 649ms/step - loss: 0.0013 - accuracy: 1.00 - 12s 618ms/step - loss: 0.0013 - accuracy: 1.00 - 12s 5